In [1]:
import pandas as pd
import numpy as np
import gc; gc.enable()
import lightgbm as lgb
import warnings 
warnings.filterwarnings('ignore')

from GridSearcher import data_loader, model_loader, fit_params, get_oof_predictions

In [2]:
DATA_TYPE = 'sparse'
DATA_NORMALIZE = True
IS_REGRESSION = True
SEED = 519

In [3]:
dl = data_loader(data_type=DATA_TYPE, is_regression=IS_REGRESSION, is_train=True, is_pure=True)
train_X, train_y = dl.load()

Arguments: sparse True True
target loaded


In [4]:
dl_test = data_loader(data_type=DATA_TYPE, is_regression=IS_REGRESSION, is_train=False, is_pure=True)
test_X = dl_test.load()

Arguments: sparse True False
target loaded


In [5]:
train_X.shape, train_y.shape, test_X.shape

((1503424, 38406), (1503424,), (508438, 38406))

In [6]:
train_y

array([0.    , 0.    , 0.    , ..., 0.7376, 0.    , 0.7376])

In [7]:
del train_X, test_X

In [8]:
import pickle
with open('reduced_sparse_features.pickle', 'rb') as handle:
    all_features = pickle.load(handle)

In [9]:
train_X, test_X = all_features[:len(train_y), :], all_features[len(train_y):, :]

In [10]:
del all_features
train_X.shape, test_X.shape

((1503424, 300), (508438, 300))

In [11]:
gc.collect()

0

## Do k-means to create groups

In [12]:
from sklearn.cluster import MiniBatchKMeans

In [13]:
group_num = 30

In [14]:
km = MiniBatchKMeans(n_clusters=group_num, batch_size=100000, verbose=1000, random_state=877, reassignment_ratio=0.6)
train_group = km.fit_predict(train_X)

Init 1/3 with method: k-means++
Inertia for init 1/3: 1831370.205926
Init 2/3 with method: k-means++
Inertia for init 2/3: 1827829.293101
Init 3/3 with method: k-means++
Inertia for init 3/3: 1799955.195155
Minibatch iteration 1/1600: mean batch inertia: 5.997702, ewa inertia: 5.997702 
Minibatch iteration 2/1600: mean batch inertia: 5.917720, ewa inertia: 5.987062 
Minibatch iteration 3/1600: mean batch inertia: 5.869316, ewa inertia: 5.971399 
Minibatch iteration 4/1600: mean batch inertia: 5.848887, ewa inertia: 5.955101 
Minibatch iteration 5/1600: mean batch inertia: 5.827393, ewa inertia: 5.938112 
Minibatch iteration 6/1600: mean batch inertia: 5.814473, ewa inertia: 5.921664 
Minibatch iteration 7/1600: mean batch inertia: 5.795214, ewa inertia: 5.904843 
Minibatch iteration 8/1600: mean batch inertia: 5.790057, ewa inertia: 5.889573 
Minibatch iteration 9/1600: mean batch inertia: 5.788202, ewa inertia: 5.876087 
Minibatch iteration 10/1600: mean batch inertia: 5.774070, ewa i

In [15]:
test_group = km.predict(test_X)

Computing label assignment and total inertia


In [16]:
train_group.shape, test_group.shape

((1503424,), (508438,))

## Within-group KNN in TF

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
from tqdm import tqdm

ops.reset_default_graph()

In [18]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 122720747942679054
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3174131302
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1450885511418735386
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [19]:
k = 50
DEBUG = False

In [20]:
x_data_train = tf.placeholder(shape=[None, train_X.shape[1]], dtype=tf.float32)
x_data_test = tf.placeholder(shape=[None, train_X.shape[1]], dtype=tf.float32)

y_target_train = tf.placeholder(shape=[None, 1], dtype=tf.float32)
y_target_test = tf.placeholder(shape=[None, 1], dtype=tf.float32)

In [21]:
#L1 
#distance = tf.reduce_sum(tf.abs(tf.subtract(x_data_train, tf.expand_dims(x_data_test,1))), axis=2)

#L2
distance = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(x_data_train, tf.expand_dims(x_data_test,1))), axis=2))

In [22]:
top_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)

In [23]:
def get_predictions(tr_x, te_x, batch_size=2, is_removed_first=False):
    num_loops = int(np.ceil(te_x.shape[0]/batch_size))

    k_xvals, k_indices = [], []
    
    for i in tqdm(range(num_loops)):
        min_index = i*batch_size
        max_index = min((i+1)*batch_size, te_x.shape[0])
        x_batch = te_x[min_index:max_index, :]
    
        if DEBUG:
            print('Distances')
            print(sess.run(distance, 
                                    feed_dict={
                                        x_data_train: tr_x, x_data_test: x_batch
                                    }))
        
        
        k_xval, k_indice = sess.run([top_k_xvals, top_k_indices], 
                                    feed_dict={
                                        x_data_train: tr_x, x_data_test: x_batch
                                    })
        
        k_xval = np.array(k_xval)
        k_indice = np.array(k_indice)
        
        if is_removed_first:
            k_xvals.extend(k_xval[:,1:]*-1)
            k_indices.extend(k_indice[:,1:])
        else:
            k_xvals.extend(k_xval*-1)
            k_indices.extend(k_indice)
            
        if DEBUG:
            print(k_xval, k_indice, k_xvals, k_indices)
            
        del k_xval, k_indice; gc.collect()
        
    return k_xvals, k_indices

## By cluster

In [24]:
from sklearn.model_selection import KFold

for group in range(group_num):
    train_filter = train_group == group
    test_filter = test_group == group
    
    tr_X = train_X[train_filter,:]
    te_X = test_X[test_filter,:]
    print(tr_X.shape, te_X.shape)
    
    if tr_X.shape[0] > 80000:
        batch_size = 2
    else:
        batch_size = 32
        
    train_dist, train_ind = get_predictions(tr_X, tr_X, batch_size=batch_size, is_removed_first=True)
    test_dist, test_ind = get_predictions(tr_X, te_X, batch_size=batch_size)
    
    with open('knn_dist_and_ind_group_{}.data'.format(group), 'wb') as handle:
        pickle.dump(
            {
                'train_filter': train_filter,
                'test_filter': test_filter,
                'train_dist': train_dist, 
                'train_ind': train_ind,
                'test_dist': test_dist,
                'test_ind': test_ind
            }, handle)
      
    del train_filter, test_filter, train_dist, train_ind, test_dist, test_ind; gc.collect()

(33007, 300) (12547, 300)


100%|████████████████████████████████████████████████████████████████████████████████| 393/393 [00:54<00:00,  7.17it/s]


(74433, 300) (25166, 300)


100%|████████████████████████████████████████████████████████████████████████████████| 787/787 [03:25<00:00,  3.83it/s]


(49632, 300) (15207, 300)


...

100%|████████████████████████████████████████████████████████████████████████████| 14864/14864 [32:15<00:00,  7.68it/s]


(30952, 300) (12312, 300)


100%|████████████████████████████████████████████████████████████████████████████████| 385/385 [00:50<00:00,  7.55it/s]


(38490, 300) (13385, 300)


100%|████████████████████████████████████████████████████████████████████████████████| 419/419 [01:04<00:00,  6.49it/s]


## Transform into knn features

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import gc; gc.enable()

In [2]:
train_X = pd.read_csv('train.csv', parse_dates=['activation_date']).sort_values('activation_date').reset_index(drop=True)
test_X = pd.read_csv('test.csv', parse_dates=['activation_date'])

In [3]:
class knn_feature_wrapper():
    def __init__(self):
        pass
        
    def transform(self, X, test_X, # X, test_X are dense, and X should contain all the feature_cols
                  group_num=30,
                  n_list=[5, 10, 20, 40],
                  #feature_inds=[38358, 38359],
                  feature_cols=['price', 'item_seq_number', 'deal_probability']):
        def get_model():
            return model_loader(model_type='knn').load(self.params)
        
        def get_feature_names(n):
            return ['{}_{}_closest_mean'.format(feat, n) for feat in feature_cols]
        
        train_res = pd.DataFrame()
        test_res = pd.DataFrame()
        for n in n_list:
            train_res['{}_closest_avg_dist'.format(n)] = np.zeros((X.shape[0],))
            test_res['{}_closest_avg_dist'.format(n)] = np.zeros((test_X.shape[0],))
            for feat in feature_cols:
                train_res['{}_{}_closest_mean'.format(feat, n)] = np.zeros((X.shape[0],))
                test_res['{}_{}_closest_mean'.format(feat, n)] = np.zeros((test_X.shape[0],))
        
        for group in range(group_num):
            with open('knn_dist_and_ind_group_{}.data'.format(group), 'rb') as handle:
                info = pickle.load(handle)
                '''
                    {
                        'train_filter': train_filter,
                        'test_filter': test_filter,
                        'train_dist': train_dist, 
                        'train_ind': train_ind,
                        'test_dist': test_dist,
                        'test_ind': test_ind
                    }
                '''

                tr_ix = np.arange(X.shape[0])[info['train_filter']]
                te_ix = np.arange(test_X.shape[0])[info['test_filter']]
                tr_X = X.loc[tr_ix, :].reset_index(drop=True)
                
                for n in n_list:
                    print('Processing n=', n)
                    
                    train_dist, train_ind = np.array(info['train_dist'])[:, :n], np.array(info['train_ind'])[:, :n]
                    test_dist, test_ind = np.array(info['test_dist'])[:, :n], np.array(info['test_ind'])[:, :n]
                
                    train_res.loc[tr_ix, '{}_closest_avg_dist'.format(n)] = train_dist.mean(axis=1)
                    train_res.loc[tr_ix, get_feature_names(n)] = \
                        np.array([tr_X.loc[ind, feature_cols].mean(axis=0) for ind in tqdm(train_ind)])
                    
                    test_res.loc[te_ix, '{}_closest_avg_dist'.format(n)] = test_dist.mean(axis=1)
                    test_res.loc[te_ix, get_feature_names(n)] = \
                        np.array([tr_X.loc[ind, feature_cols].mean(axis=0) for ind in tqdm(test_ind)])
                    
                del tr_ix, te_ix, tr_X, info; gc.collect()
        
        return train_res, test_res

In [4]:
kfw = knn_feature_wrapper()
train_res, test_res = kfw.transform(train_X, test_X)

Processing n= 5


100%|███████████████████████████████████████████████████████████████████████████| 12547/12547 [00:17<00:00, 722.06it/s]


Processing n= 10


100%|███████████████████████████████████████████████████████████████████████████| 12547/12547 [00:17<00:00, 714.64it/s]


Processing n= 20


...

100%|███████████████████████████████████████████████████████████████████████████| 13385/13385 [00:19<00:00, 680.97it/s]


Processing n= 20


100%|███████████████████████████████████████████████████████████████████████████| 13385/13385 [00:19<00:00, 682.18it/s]


Processing n= 40


100%|███████████████████████████████████████████████████████████████████████████| 13385/13385 [00:19<00:00, 685.56it/s]


In [5]:
train_res.head(3)

,5_closest_avg_dist,price_5_closest_mean,item_seq_number_5_closest_mean,deal_probability_5_closest_mean,10_closest_avg_dist,price_10_closest_mean,item_seq_number_10_closest_mean,deal_probability_10_closest_mean,20_closest_avg_dist,price_20_closest_mean,item_seq_number_20_closest_mean,deal_probability_20_closest_mean,40_closest_avg_dist,price_40_closest_mean,item_seq_number_40_closest_mean,deal_probability_40_closest_mean
0,0.996793,1320.0,32.8,0.000000,1.064976,1000.0,42.4,0.000000,1.146944,975.0,59.75,0.000000,1.272664,1014.0,49.625,0.043200
1,0.854518,230.0,50.2,0.153572,1.051065,280.0,61.4,0.076786,1.243759,250.0,48.55,0.046991,1.429779,285.5,39.525,0.044163
2,1.122214,7840.0,8.2,0.000000,1.247133,4500.0,9.8,0.031626,1.376999,4125.0,7.20,0.031626,1.571354,4687.5,7.675,0.029212


In [6]:
with open('knn_train_features.pickle', 'wb') as handle:
    pickle.dump(train_res, handle)
    
with open('knn_test_features.pickle', 'wb') as handle:
    pickle.dump(test_res, handle)

In [7]:
y = pd.read_csv('regression_target.csv')['deal_probability'].values

In [9]:
train_res.loc[:, 'target'] = y

In [10]:
train_res.corr()

,5_closest_avg_dist,price_5_closest_mean,item_seq_number_5_closest_mean,deal_probability_5_closest_mean,10_closest_avg_dist,price_10_closest_mean,item_seq_number_10_closest_mean,deal_probability_10_closest_mean,20_closest_avg_dist,price_20_closest_mean,item_seq_number_20_closest_mean,deal_probability_20_closest_mean,40_closest_avg_dist,price_40_closest_mean,item_seq_number_40_closest_mean,deal_probability_40_closest_mean,target
5_closest_avg_dist,1.000000,-0.014205,-0.091738,-0.093835,0.984972,-0.003515,-0.093842,-0.109662,0.951325,-0.004601,-0.095569,-0.122972,0.909109,-0.004771,-0.097470,-0.133854,-0.060841
price_5_closest_mean,-0.014205,1.000000,0.160624,-0.028159,-0.012218,0.082910,0.165432,-0.030406,-0.008660,0.087181,0.167944,-0.031530,-0.005000,0.092743,0.169840,-0.031789,-0.016269
item_seq_number_5_closest_mean,-0.091738,0.160624,1.000000,-0.064070,-0.094051,0.018640,0.979368,-0.073117,-0.094503,0.026802,0.958159,-0.080375,-0.092301,0.035174,0.936886,-0.085959,-0.039176
deal_probability_5_closest_mean,-0.093835,-0.028159,-0.064070,1.000000,-0.097554,-0.003458,-0.066003,0.886700,-0.099862,-0.004727,-0.068139,0.818125,-0.100409,-0.006400,-0.070700,0.776596,0.377628
10_closest_avg_dist,0.984972,-0.012218,-0.094051,-0.097554,1.000000,-0.002598,-0.096300,-0.113350,0.986737,-0.003454,-0.097947,-0.126942,0.955400,-0.003838,-0.099742,-0.138147,-0.063865
price_10_closest_mean,-0.003515,0.082910,0.018640,-0.003458,-0.002598,1.000000,0.019258,-0.003292,-0.001868,0.709073,0.019564,-0.003589,-0.001346,0.475245,0.019810,-0.003667,-0.002088
item_seq_number_10_closest_mean,-0.093842,0.165432,0.979368,-0.066003,-0.096300,0.019258,1.000000,-0.075383,-0.096998,0.027741,0.983081,-0.082947,-0.094846,0.036402,0.961447,-0.088673,-0.040340
deal_probability_10_closest_mean,-0.109662,-0.030406,-0.073117,0.886700,-0.113350,-0.003292,-0.075383,1.000000,-0.115180,-0.004587,-0.077770,0.925186,-0.115424,-0.006457,-0.080726,0.878968,0.411679
20_closest_avg_dist,0.951325,-0.008660,-0.094503,-0.099862,0.986737,-0.001868,-0.096998,-0.115180,1.000000,-0.002570,-0.098805,-0.128414,0.987456,-0.002957,-0.100496,-0.139762,-0.065458
price_20_closest_mean,-0.004601,0.087181,0.026802,-0.004727,-0.003454,0.709073,0.027741,-0.004587,-0.002570,1.000000,0.028398,-0.005258,-0.001909,0.670382,0.028801,-0.005464,-0.002996


## By fold: (Warining: Will runs really long)

In [28]:
from sklearn.model_selection import KFold
kf = KFold(5, shuffle=True, random_state=SEED)

fold=1
for train_ix, val_ix in kf.split(train_X):
    print('Processing fold # =', fold)
    
    train_ix = np.random.choice(train_ix, len(val_ix), replace=False)
    tr_X, tr_y = train_X[train_ix,:], train_y[train_ix]
    val_X = train_X[val_ix,:]
        
    val_dist, val_ind = get_predictions(tr_X, val_X)
    test_dist, test_ind = get_predictions(tr_X, test_X)
    
    with open('knn_dist_and_ind_fold_{}.data'.format(fold), 'wb') as handle:
        pickle.dump(
            {
                'train_ix': train_ix,
                'val_dist': val_dist, 
                'val_ind': val_ind,
                'test_dist': test_dist,
                'test_ind': test_ind
            }, handle)
    
    del val_dist, val_ind, test_dist, test_ind; gc.collect()
    fold += 1

Processing fold # = 1


  0%|                                                                           | 79/300685 [00:25<26:58:56,  3.09it/s]

KeyboardInterrupt: 